In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 24: Planet of Discord ---</h2><p>You land on <a href="https://en.wikipedia.org/wiki/Eris_(dwarf_planet)">Eris</a>, your last stop before reaching Santa.  As soon as you do, your sensors start picking up strange life forms moving around: Eris is infested with <a href="https://www.nationalgeographic.org/thisday/sep9/worlds-first-computer-bug/">bugs</a>! With an <span title="For a sad version of this story, look up Voices of a Distant Star.">over 24-hour roundtrip</span> for messages between you and Earth, you'll have to deal with this problem on your own.</p>
<p>Eris isn't a very large place; a scan of the entire area fits into a 5x5 grid (your puzzle input). The scan shows <em>bugs</em> (<code>#</code>) and <em>empty spaces</em> (<code>.</code>).</p>
<p>Each <em>minute</em>, The bugs live and die based on the number of bugs in the <em>four adjacent tiles</em>:</p>
<ul>
<li>A bug <em>dies</em> (becoming an empty space) unless there is <em>exactly one</em> bug adjacent to it.</li>
<li>An empty space <em>becomes infested</em> with a bug if <em>exactly one or two</em> bugs are adjacent to it.</li>
</ul>
<p>Otherwise, a bug or empty space remains the same.  (Tiles on the edges of the grid have fewer than four adjacent tiles; the missing tiles count as empty space.) This process happens in every location <em>simultaneously</em>; that is, within the same minute, the number of adjacent bugs is counted for every tile first, and then the tiles are updated.</p>
<p>Here are the first few minutes of an example scenario:</p>
<pre><code>Initial state:
....#
#..#.
#..##
..#..
#....

After 1 minute:
#..#.
####.
###.#
##.##
.##..

After 2 minutes:

#####

....#
....#
...#.
#.###

After 3 minutes:
#....
####.
...##
#.##.
.##.#

After 4 minutes:
####.
....#
##..#
.....
##...
</code></pre>

<p>To understand the nature of the bugs, watch for the first time a layout of bugs and empty spaces <em>matches any previous layout</em>. In the example above, the first layout to appear twice is:</p>
<pre><code>.....
.....
.....
#....
.#...
</code></pre>
<p>To calculate the <em>biodiversity rating</em> for this layout, consider each tile left-to-right in the top row, then left-to-right in the second row, and so on. Each of these tiles is worth biodiversity points equal to <em>increasing powers of two</em>: 1, 2, 4, 8, 16, 32, and so on.  Add up the biodiversity points for tiles with bugs; in this example, the 16th tile (<code>32768</code> points) and 22nd tile (<code>2097152</code> points) have bugs, a total biodiversity rating of <code><em>2129920</em></code>.</p>
<p><em>What is the biodiversity rating for the first layout that appears twice?</em></p>
</article>


In [2]:
from copy import deepcopy
from enum import Enum
from itertools import chain
from math import prod

from numpy import empty


example = """
    ....#
    #..#.
    #..##
    ..#..
    #....
"""


class Eris:
    adjacent = ((-1, 0), (0, 1), (1, 0), (0, -1))

    def __init__(self, s: str) -> None:
        self.grid = [[t == "#" for t in l.strip()] for l in s.strip().splitlines()]

    def minute(self) -> Eris:
        new_e = deepcopy(self)
        rows, cols = len(self.grid), len(self.grid[0])

        for r, c in product(range(rows), range(cols)):
            bugs_adjacent = sum(
                self.grid[r + dr][c + dc]
                for dr, dc in self.adjacent
                if 0 <= r + dr < rows and 0 <= c + dc < cols
            )
            if self.grid[r][c]:
                new_e.grid[r][c] = bugs_adjacent == 1
            else:
                new_e.grid[r][c] = bugs_adjacent == 1 or bugs_adjacent == 2

        return new_e

    @classmethod
    def get_first_repeating_layout(cls, s: str) -> Eris:
        current = cls(s)
        seen = set()

        while True:
            hsh = hash(current)
            if hsh in seen:
                return current
            seen.add(hsh)
            current = current.minute()

    def biodiversity_rating(self) -> int:
        return hash(self)

    def __hash__(self) -> int:
        return sum(int(t) * 2**i for i, t in enumerate(chain.from_iterable(self.grid)))

    def __repr__(self) -> str:
        return "\n".join("".join("#" if t else "." for t in l) for l in self.grid)


assert Eris.get_first_repeating_layout(example).biodiversity_rating() == 2129920

In [3]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

print(f"Part I: {Eris.get_first_repeating_layout(puzzle).biodiversity_rating()}")

Part I: 12129040


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>12129040</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>After careful analysis, one thing is certain: <em>you have no idea where all these bugs are coming from</em>.</p>
<p>Then, you remember: Eris is an old <a href="20">Plutonian</a> settlement! Clearly, the bugs are coming from recursively-folded space.</p>
<p>This 5x5 grid is <em>only one</em> level in an <em>infinite</em> number of recursion levels. The tile in the middle of the grid is actually another 5x5 grid, the grid in your scan is contained as the middle tile of a larger 5x5 grid, and so on. Two levels of grids look like this:</p>
<pre><code>     |     |         |     |     
     |     |         |     |     
     |     |         |     |     
-----+-----+---------+-----+-----
     |     |         |     |     
     |     |         |     |     
     |     |         |     |     
-----+-----+---------+-----+-----
     |     | | | | | |     |     
     |     |-+-+-+-+-|     |     
     |     | | | | | |     |     
     |     |-+-+-+-+-|     |     
     |     | | |?| | |     |     
     |     |-+-+-+-+-|     |     
     |     | | | | | |     |     
     |     |-+-+-+-+-|     |     
     |     | | | | | |     |     
-----+-----+---------+-----+-----
     |     |         |     |     
     |     |         |     |     
     |     |         |     |     
-----+-----+---------+-----+-----
     |     |         |     |     
     |     |         |     |     
     |     |         |     |     
</code></pre>
<p>(To save space, some of the tiles are not drawn to scale.)  Remember, this is only a small part of the infinitely recursive grid; there is a 5x5 grid that contains this diagram, and a 5x5 grid that contains that one, and so on.  Also, the <code>?</code> in the diagram contains another 5x5 grid, which itself contains another 5x5 grid, and so on.</p>
<p>The scan you took (your puzzle input) shows where the bugs are <em>on a single level</em> of this structure. The middle tile of your scan is empty to accommodate the recursive grids within it. Initially, no other levels contain bugs.</p>
<p>Tiles still count as <em>adjacent</em> if they are directly <em>up, down, left, or right</em> of a given tile. Some tiles have adjacent tiles at a recursion level above or below its own level. For example:</p>
<pre><code>     |     |         |     |     
  1  |  2  |    3    |  4  |  5  
     |     |         |     |     
-----+-----+---------+-----+-----
     |     |         |     |     
  6  |  7  |    8    |  9  |  10 
     |     |         |     |     
-----+-----+---------+-----+-----
     |     |A|B|C|D|E|     |     
     |     |-+-+-+-+-|     |     
     |     |F|G|H|I|J|     |     
     |     |-+-+-+-+-|     |     
 11  | 12  |K|L|?|N|O|  14 |  15 
     |     |-+-+-+-+-|     |     
     |     |P|Q|R|S|T|     |     
     |     |-+-+-+-+-|     |     
     |     |U|V|W|X|Y|     |     
-----+-----+---------+-----+-----
     |     |         |     |     
 16  | 17  |    18   |  19 |  20 
     |     |         |     |     
-----+-----+---------+-----+-----
     |     |         |     |     
 21  | 22  |    23   |  24 |  25 
     |     |         |     |     
</code></pre>
<ul>
<li>Tile 19 has four adjacent tiles: 14, 18, 20, and 24.</li>
<li>Tile G has four adjacent tiles: B, F, H, and L.</li>
<li>Tile D has four adjacent tiles: 8, C, E, and I.</li>
<li>Tile E has four adjacent tiles: 8, D, 14, and J.</li>
<li>Tile 14 has <em>eight</em> adjacent tiles: 9, E, J, O, T, Y, 15, and 19.</li>
<li>Tile N has <em>eight</em> adjacent tiles: I, O, S, and five tiles within the sub-grid marked <code>?</code>.</li>
</ul>
<p>The rules about bugs living and dying are the same as before.</p>
<p>For example, consider the same initial state as above:</p>
<pre><code>....#
#..#.
#.?##
..#..
#....
</code></pre>
<p>The center tile is drawn as <code>?</code> to indicate the next recursive grid. Call this level 0; the grid within this one is level 1, and the grid that contains this one is level -1.  Then, after <em>ten</em> minutes, the grid at each level would look like this:</p>
<pre><code>Depth -5:
..#..
.#.#.
..?.#
.#.#.
..#..

Depth -4:
...#.
...##
..?..
...##
...#.

Depth -3:
#.#..
.#...
..?..
.#...
#.#..

Depth -2:
.#.##
....#
..?.#
...##
.###.

Depth -1:
#..##
...##
..?..
...#.
.####

Depth 0:
.#...
.#.##
.#?..
.....
.....

Depth 1:
.##..
#..##
..?.#
##.##

#####

Depth 2:
###..
##.#.
#.?..
.#.##
#.#..

Depth 3:
..###
.....
#.?..
#....
#...#

Depth 4:
.###.
#..#.
#.?..
##.#.
.....

Depth 5:
####.
#..#.
#.?#.
####.
.....
</code></pre>

<p>In this example, after 10 minutes, a total of <code><em>99</em></code> bugs are present.</p>
<p>Starting with your scan, <em>how many bugs are present after 200 minutes?</em></p>
</article>

</main>


In [4]:
from typing import Iterator


class RecursiveSpace:
    adjacent = ((-1, 0), (0, 1), (1, 0), (0, -1))

    def __init__(self, s: str = None, level: int = 0) -> None:
        self.level = level
        self.bugs = 0

        if s is None:
            self.grid = [[0] * 5 for _ in range(5)]
        else:
            self.grid = [
                [int(t == "#") for t in l.strip()] for l in s.strip().splitlines()
            ]
            self.bugs = sum(
                1
                for r, c in product(range(5), repeat=2)
                if (r != 2 or c != 2) and self.grid[r][c]
            )

        self.grid[2][2] = 2

        self.level_down = None
        self.level_up = None
        if s:
            self.level_down = RecursiveSpace(level=level - 1)
            self.level_down.level_up = self
            self.level_up = RecursiveSpace(level=level + 1)
            self.level_up.level_down = self

    def get_neigbor_values(self, r: int, c: int) -> Iterator[int]:
        if r == 2 and c == 2:
            return

        if self.level_down is not None:
            if r == 0:
                yield self.level_down.grid[1][2]
            if c == 4:
                yield self.level_down.grid[2][3]
            if r == 4:
                yield self.level_down.grid[3][2]
            if c == 0:
                yield self.level_down.grid[2][1]

        yield from (
            self.grid[r + dr][c + dc]
            for dr, dc in self.adjacent
            if 0 <= r + dr < 5 and 0 <= c + dc < 5 and (r + dr != 2 or c + dc != 2)
        )

        if self.level_up is not None:
            if r == 1 and c == 2:
                yield from (self.level_up.grid[0][cc] for cc in range(5))
            if r == 2 and c == 3:
                yield from (self.level_up.grid[rr][4] for rr in range(5))
            if r == 3 and c == 2:
                yield from (self.level_up.grid[4][cc] for cc in range(5))
            if r == 2 and c == 1:
                yield from (self.level_up.grid[rr][0] for rr in range(5))

    def minute(self) -> RecursiveSpace:
        new_rc = RecursiveSpace(level=self.level)
        bugs = 0
        for r, c in product(range(5), repeat=2):
            if r == 2 and c == 2:
                new_rc.grid[r][c] = self.grid[r][c]
            else:
                bugs_adjacent = sum(self.get_neigbor_values(r, c))
                if self.grid[r][c]:
                    new_rc.grid[r][c] = bugs_adjacent == 1
                else:
                    new_rc.grid[r][c] = bugs_adjacent == 1 or bugs_adjacent == 2

                if new_rc.grid[r][c]:
                    bugs += 1

        if self.level <= 0:
            if self.level_down is None:
                new_rc.level_down = RecursiveSpace(level=self.level - 1)
                new_rc.level_down.level_up = new_rc
            else:
                new_rc.level_down = self.level_down.minute()
                new_rc.level_down.level_up = new_rc
                bugs += new_rc.level_down.bugs

        if self.level >= 0:
            if self.level_up is None:
                new_rc.level_up = RecursiveSpace(level=self.level + 1)
                new_rc.level_up.level_down = new_rc
            else:
                new_rc.level_up = self.level_up.minute()
                new_rc.level_up.level_down = new_rc
                bugs += new_rc.level_up.bugs

        new_rc.bugs = bugs

        return new_rc

    def __repr__(self) -> str:
        tile_to_str = {0: ".", 1: "#", 2: "?"}
        sb = [
            f"{self.level_down}" if self.level <= 0 else "",
            "",
            f"depth={self.level},",
            "",
            *["".join(tile_to_str[t] for t in l) for l in self.grid],
            "",
            "",
            f"{self.level_up}" if self.level >= 0 else "",
        ]
        return "\n".join(sb)

    @classmethod
    def bugs_after_minutes(cls, s: str, m: int) -> int:
        current = cls(s)

        for _ in range(m):
            current = current.minute()

        return current.bugs


assert RecursiveSpace.bugs_after_minutes(example, 10) == 99

In [5]:
print(f"Part II: {RecursiveSpace.bugs_after_minutes(puzzle, 200)}")

Part II: 2109


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2109</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
